In [75]:
import csv
import re
from os import walk

In [76]:
files = []
for (dirpath, dirnames, filenames) in walk('./input/'):
    files.extend(filenames)
files.remove('README.md')
    
with open('./input/' + files[0]) as f:
    lines = list(csv.reader(f, delimiter=';'))
files

['tst.csv']

In [77]:
lines[0]

['',
 'CASE_NUMBER',
 'CASE_STATUS',
 'CASE_SUBMITTED',
 'DECISION_DATE',
 'VISA_CLASS',
 'EMPLOYMENT_START_DATE',
 'EMPLOYMENT_END_DATE',
 'EMPLOYER_NAME',
 'EMPLOYER_BUSINESS_DBA',
 'EMPLOYER_ADDRESS',
 'EMPLOYER_CITY',
 'EMPLOYER_STATE',
 'EMPLOYER_POSTAL_CODE',
 'EMPLOYER_COUNTRY',
 'EMPLOYER_PROVINCE',
 'EMPLOYER_PHONE',
 'EMPLOYER_PHONE_EXT',
 'AGENT_REPRESENTING_EMPLOYER',
 'AGENT_ATTORNEY_NAME',
 'AGENT_ATTORNEY_CITY',
 'AGENT_ATTORNEY_STATE',
 'JOB_TITLE',
 'SOC_CODE',
 'SOC_NAME',
 'NAICS_CODE',
 'TOTAL_WORKERS',
 'NEW_EMPLOYMENT',
 'CONTINUED_EMPLOYMENT',
 'CHANGE_PREVIOUS_EMPLOYMENT',
 'NEW_CONCURRENT_EMP',
 'CHANGE_EMPLOYER',
 'AMENDED_PETITION',
 'FULL_TIME_POSITION',
 'PREVAILING_WAGE',
 'PW_UNIT_OF_PAY',
 'PW_WAGE_LEVEL',
 'PW_SOURCE',
 'PW_SOURCE_YEAR',
 'PW_SOURCE_OTHER',
 'WAGE_RATE_OF_PAY_FROM',
 'WAGE_RATE_OF_PAY_TO',
 'WAGE_UNIT_OF_PAY',
 'H1B_DEPENDENT',
 'WILLFUL_VIOLATOR',
 'SUPPORT_H1B',
 'LABOR_CON_AGREE',
 'PUBLIC_DISCLOSURE_LOCATION',
 'WORKSITE_CITY',
 'WO

In [78]:
lines[0][1]

'CASE_NUMBER'

In [79]:
def header_index(arg, header):
    for i, ele in enumerate(header):
        if re.match(arg, ele):
            return i
    raise NameError('column not found')

def populate_indices(header):
    indices = {}
    indices['status'] = header_index('.*STATUS.*', header)
    indices['soc'] = header_index('.*SOC_CODE.*', header)
    indices['job_name'] = header_index('.*SOC_NAME.*', header)
    indices['job_state'] = header_index('.*(WORKSITE_STATE|WORKLOC1_STATE).*', header)
    return indices    

In [80]:
indices = populate_indices(lines[0])
indices

{'status': 2, 'soc': 23, 'job_name': 24, 'job_state': 50}

In [81]:
def check_soc_hyphen(soc):
    if len(soc) < 6:
        return soc
    soc_lst = list(soc)
    
    if soc_lst[2] != '-':
        if soc_lst[2] == '.':
            soc_lst[2] = '-'
        else:
            soc_lst.insert(2, '-')
    return ''.join(soc_lst)

def clean_soc(soc):
    soc_strip = soc.replace(' ', '')
    soc_hyphen = check_soc_hyphen(soc_strip)
    return soc_hyphen[:7]     

In [82]:
occupation_dict = {}
occupation_count = {}
state_count = {}
certified_count = 0

for line in lines[1:]:
    if line[indices['status']] != 'CERTIFIED':
        continue    
    
    soc = clean_soc(line[indices['soc']])  
    job_name = line[indices['job_name']]
    job_state = line[indices['job_state']]
    if not re.match('\d{2}-\d{4}', soc) or not job_state:
        continue
        
    certified_count += 1
    
    if soc not in occupation_dict:
        occupation_dict[soc] = job_name
        occupation_count[soc] = 0
    elif len(job_name) > len(occupation_dict[soc]):
        occupation_dict[soc] = job_name
    occupation_count[soc] += 1
    
    state = line[indices['job_state']]
    if state not in state_count:
        state_count[state] = 0
    state_count[state] += 1
    

In [83]:
occupation_dict

{'15-1132': 'SOFTWARE DEVELOPERS, APPLICATIONS',
 '13-2011': 'ACCOUNTANTS AND AUDITORS',
 '15-1141': 'DATABASE ADMINISTRATORS',
 '15-1121': 'COMPUTER SYSTEMS ANALYST',
 '15-1199': 'COMPUTER OCCUPATIONS, ALL OTHER'}

In [84]:
occupation_count

{'15-1132': 6, '13-2011': 1, '15-1141': 1, '15-1121': 1, '15-1199': 1}

In [85]:
print(len(state_count))
state_count

9


{'WA': 1,
 'CA': 1,
 'TX': 1,
 'DE': 1,
 'AL': 1,
 'FL': 2,
 'MD': 1,
 'NJ': 1,
 'GA': 1}

In [86]:
certified_count

10

In [87]:
def ratio_formatted(job_count, total):
    num = round(100 * float(job_count) / total, 1)
    return str(num) + '%'

def occupation_analysis(count, names, total):
    sorted_occupations = sorted(count.items(), key=lambda kv: kv[1])
    sorted_occupations.reverse()
    top_ten = sorted_occupations[:10]
    
    occupation_answer = ['TOP_OCCUPATIONS;NUMBER_CERTIFIED_APPLICATIONS;PERCENTAGE']
    for job in top_ten:
        row = []
        row.append(names[job[0]])
        row.append(str(job[1]))
        row.append(ratio_formatted(job[1], total))
        occupation_answer.append(';'.join(row))
    return "\n".join(occupation_answer)
        

In [88]:
print(occupation_analysis(occupation_count, occupation_dict, certified_count))

TOP_OCCUPATIONS;NUMBER_CERTIFIED_APPLICATIONS;PERCENTAGE
SOFTWARE DEVELOPERS, APPLICATIONS;6;60.0%
COMPUTER OCCUPATIONS, ALL OTHER;1;10.0%
COMPUTER SYSTEMS ANALYST;1;10.0%
DATABASE ADMINISTRATORS;1;10.0%
ACCOUNTANTS AND AUDITORS;1;10.0%


In [89]:
def state_analysis(count, total):
    sorted_states = sorted(count.items(), key=lambda kv: kv[1])
    sorted_states.reverse()
    top_ten = sorted_states[:10]
    
    states_answer = ['TOP_STATES;NUMBER_CERTIFIED_APPLICATIONS;PERCENTAGE']
    for state in top_ten:
        row = []
        row.append(state[0])
        row.append(str(state[1]))
        row.append(ratio_formatted(state[1], total))
        states_answer.append(';'.join(row))
    return "\n".join(states_answer)

In [90]:
print(state_analysis(state_count, certified_count))

TOP_STATES;NUMBER_CERTIFIED_APPLICATIONS;PERCENTAGE
FL;2;20.0%
GA;1;10.0%
NJ;1;10.0%
MD;1;10.0%
AL;1;10.0%
DE;1;10.0%
TX;1;10.0%
CA;1;10.0%
WA;1;10.0%
